In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

In [2]:
sc = SparkSession.builder.appName("read_mysql").getOrCreate()

In [3]:
ctx = SQLContext(sc)

In [4]:
url = 'jdbc:mysql://localhost:3306/spark_db?user=spark&password=sparkpw'
# useUnicode=true&characterEncoding=GBK

In [5]:
%%time
df_r = ctx.read.format('jdbc')\
    .option("url",url + '&rewriteBatchedStatements=true')\
    .option('fetchsize',10000)\
    .option("driver","com.mysql.jdbc.Driver")\
    .option("numPartitions",20)\
    .option("partitionColumn","shop_id")\
    .option("lowerBound", "0")\
    .option("upperBound","2000")\
    .option("dbtable", "sales22").load()

CPU times: user 3.27 ms, sys: 1.87 ms, total: 5.13 ms
Wall time: 2.05 s


In [6]:
df_r.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- date_block_num: integer (nullable = true)
 |-- shop_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- item_price: double (nullable = true)
 |-- item_cnt_day: double (nullable = true)



In [7]:
%%time
df_r.count()

CPU times: user 2.62 ms, sys: 2.35 ms, total: 4.97 ms
Wall time: 25.4 s


5871698

In [11]:
df_r.createOrReplaceTempView('sales11')

In [13]:
%%time
spark.sql("""
select * from sales11 
where shop_id=1
limit 100
""").show()

+-------------------+--------------+-------+-------+----------+------------+
|               date|date_block_num|shop_id|item_id|item_price|item_cnt_day|
+-------------------+--------------+-------+-------+----------+------------+
|2013-01-14 00:00:00|             0|      1|   8070|     137.0|         1.0|
|2013-01-20 00:00:00|             0|      1|  11415|      70.0|         1.0|
|2013-01-11 00:00:00|             0|      1|  11435|     110.0|         1.0|
|2013-01-29 00:00:00|             0|      1|  11441|      75.0|         1.0|
|2013-01-20 00:00:00|             0|      1|  11450|     249.0|         1.0|
|2013-01-19 00:00:00|             0|      1|  11454|      77.0|         1.0|
|2013-01-14 00:00:00|             0|      1|  11464|      76.0|         1.0|
|2013-01-27 00:00:00|             0|      1|  11464|      76.0|         1.0|
|2013-01-09 00:00:00|             0|      1|  11481|      69.0|         1.0|
|2013-01-10 00:00:00|             0|      1|  11481|      69.0|         1.0|

In [ ]:
%%time
df_r.write.format('orc').mode('overwrite').saveAsTable('sales')